<h2>NFL Betting Project: Create Preprocessed Game Data CSV</h2>

In [12]:
# author: Warren Blood
# last update: 12-1-2021

In [13]:
# import packages
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn import preprocessing

<h3>Data Import</h3>

In [14]:
# import game data csv file
game_data = pd.read_csv('Game Data 2013-2021.csv', sep=',', encoding='unicode_escape')

# order by GameId to place games in chronological order
game_data = game_data.sort_values('GameId',ascending=True)

# check for any missing values in data
print('Number of missing values in data: ' + str(sum(game_data.isnull().sum())))


game_data.head()

Number of missing values in data: 0


,GameId,Game Date,Time Slot,Home Team,Away Team,Divisional Matchup,Stadium,Covered Stadium,Temp,Humidity,...,Away FGA (50+),Away FGM (50+),Away Offense Penalties,Away Offense Penalty Yds,Away Defense Penalties,Away Defense Penalty Yds,Away Penalty First Downs,Vegas Spread Favorite,Vegas Spread,Vegas Over Under
0,2013090500,9/5/2013,Thu Night,DEN,BAL,0,Empower Field at Mile High,0,83,0.30,...,0,0,4,35,3,18,2,DEN,-7.5,48.0
1,2013090800,9/8/2013,Sun Early,BUF,NE,1,Highmark Stadium,0,65,0.55,...,0,0,2,6,2,30,1,NE,-10.5,50.5
2,2013090801,9/8/2013,Sun Early,CAR,SEA,0,Bank of America Stadium,0,87,0.48,...,0,0,6,66,3,33,1,SEA,-3.5,44.5
3,2013090802,9/8/2013,Sun Early,CHI,CIN,0,Soldier Field,0,73,0.79,...,0,0,5,54,3,30,1,CHI,-3.0,41.5
4,2013090803,9/8/2013,Sun Early,CLE,MIA,0,FirstEnergy Stadium,0,70,0.80,...,0,0,4,25,6,52,1,CLE,-2.5,40.5


In [15]:
# n_game_stats: number of most recent games over which to evaluate team stats (between 5 and 10)
n_game_stats = 6

# start_game_index: index row of game_data containing the first game with available team stats
# add 1 to n_game_stats to account for bye week
start_game_index = game_data.index[game_data['Time Slot']=='Thu Night'].tolist()[n_game_stats + 1]

# num_total_games: number of games available with team stats
num_total_games = game_data.shape[0] - start_game_index

# avail_game_data: all games in game_data with team stats available
avail_game_data = game_data[start_game_index:]


print('Number of available games in data: ' + str(num_total_games))

Number of available games in data: 2008


<h3>Function: Get Previous Games Indexes</h3>

In [16]:
# return a list of indexes corresponding to the previous (n_game_stats) games in game_data that a team played
# before a given game (gameId) 
def get_previous_games_i(team, gameId):
    last_game = game_data.index[game_data['GameId']==gameId][0]
    previous_games = game_data[:last_game]
    previous_games_i = []
    for x in range(n_game_stats):
        if(previous_games.index[previous_games['Away Team']==team].tolist() == []):
            away_game = 0
        else:
            away_game = previous_games.index[previous_games['Away Team']==team].tolist()[-1]
        
        if(previous_games.index[previous_games['Home Team']==team].tolist() == []):
            home_game = 0
        else:
            home_game = previous_games.index[previous_games['Home Team']==team].tolist()[-1]
        
        last_game = max(away_game, home_game)
        previous_games_i.append(last_game)
        previous_games = game_data[:last_game]
        
    return previous_games_i

<h3>Data Preprocessing</h3>

In [17]:
# for one-hot encoding categorical data
enc = preprocessing.OneHotEncoder(handle_unknown='ignore')

In [18]:
# create general game features

# binary: 1 if game played in indoor stadium, 0 otherwise
indoor_stadium = avail_game_data[['Covered Stadium']].to_numpy()

# binary: 1 if game played between teams in the same division, 0 otherwise
divisional = avail_game_data[['Divisional Matchup']].to_numpy()

# one-hot encoded: for each feature, 1 if game played in a specific stadium, 0 otherwise
stadium_text = avail_game_data[['Stadium']].to_numpy()
stadium = enc.fit(stadium_text).transform(stadium_text).A

# game temperature (70 if indoor)
temp = avail_game_data[['Temp']].to_numpy()

# game humidity (0.45 if indoor)
humidity = avail_game_data[['Humidity']].to_numpy()

# game wind speed (0 if indoor)
wind_speed = avail_game_data[['Wind Speed (mph)']].to_numpy()

# create point total column: home pts + away pts
point_total = avail_game_data[['Home Pts']].to_numpy() + avail_game_data[['Away Pts']].to_numpy()

# vegas over-under line
vegas_over_under = avail_game_data[['Vegas Over Under']].to_numpy()

# binary: 1 if point total hit the vegas over, 0 otherwise
hit_over = np.zeros((len(avail_game_data),1))
for i in range(len(hit_over)):
    if(point_total[i] > vegas_over_under[i]):
        hit_over[i] = 1
    else:
        hit_over[i] = 0

# home team point differential (actual spread: home pts - away pts)
home_min_away_spread = avail_game_data[['Home Pts']].to_numpy() - avail_game_data[['Away Pts']].to_numpy()

# binary: 1 if home team is vegas spread favorite, 0 otherwise
home_vegas_favorite = np.zeros((len(avail_game_data),1))
for i in range(len(home_vegas_favorite)):
    if(avail_game_data['Vegas Spread Favorite'].to_numpy()[i] == avail_game_data['Home Team'].to_numpy()[i]):
        home_vegas_favorite[i] = 1
    else:
        home_vegas_favorite[i] = 0

# home team point differential vegas line (vegas spread: home pts - away pts)
vegas_home_min_away_spread = np.zeros((len(avail_game_data),1))
for i in range(len(vegas_home_min_away_spread)):
    if(home_vegas_favorite[i]==1):
        vegas_home_min_away_spread[i] = -1 * avail_game_data[['Vegas Spread']].to_numpy()[i]
    else:
        vegas_home_min_away_spread[i] = avail_game_data[['Vegas Spread']].to_numpy()[i]

# binary: 1 if home team covered the vegas spread, 0 otherwise
home_covered_spread = np.zeros((len(avail_game_data),1))
for i in range(len(home_covered_spread)):
    if(home_min_away_spread[i] > vegas_home_min_away_spread[i]):
        home_covered_spread[i] = 1
    else:
        home_covered_spread[i] = 0

In [19]:
# create home team features

# one-hot encoded: for each feature, 1 if home team has a specific head coach, 0 otherwise
home_coach_text = avail_game_data[['Home Coach']].to_numpy()
home_coach = enc.fit(home_coach_text).transform(home_coach_text).A

# number of days since home team's last game
home_rest_days = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    game_date = datetime.strptime(avail_game_data['Game Date'].to_numpy()[i],'%m/%d/%Y')
    previous_games = game_data[:start_game_index + i]
    last_away_game_i = previous_games.index[previous_games['Away Team']==team].tolist()[-1]
    last_home_game_i = previous_games.index[previous_games['Home Team']==team].tolist()[-1]
    last_game_i = max(last_away_game_i, last_home_game_i)
    last_game_date = datetime.strptime(game_data['Game Date'].to_numpy()[last_game_i],'%m/%d/%Y')
    home_rest_days[i] = (game_date - last_game_date).days

# home team average time of possession (not including overtime)
home_avg_pos_time = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += game_data['Home Time of Posession'][j]
        else:
            s += game_data['Away Time of Posession'][j]
    
    home_avg_pos_time[i] = round(s / n_game_stats, 2)

# home team PAT success rate
home_PAT = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_PAT_a = 0
    s_PAT_m = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_PAT_a += float(game_data['Home PAT Attempts'][j])
            s_PAT_m += float(game_data['Home PAT Made'][j])
        else:
            s_PAT_a += float(game_data['Away PAT Attempts'][j])
            s_PAT_m += float(game_data['Away PAT Made'][j])
    
    if(s_PAT_a==0):
        home_PAT[i] = 0.0
    else:
        home_PAT[i] = round(s_PAT_m / s_PAT_a, 4)

# home team field goal success rate from 0-39 yards
home_FG_39 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_fga = 0
    s_fgm = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j][0]==team):
            s_fga += float(game_data['Home FGA (0-39)'][j])
            s_fgm += float(game_data['Home FGM (0-39)'][j])
        else:
            s_fga += float(game_data['Away FGA (0-39)'][j])
            s_fgm += float(game_data['Away FGM (0-39)'][j])
    
    if(s_fga==0):
        home_FG_39[i] = 0.0
    else: 
        home_FG_39[i] = round(s_fgm / s_fga, 4)
        
# home team field goal success rate from 40-49 yards   
home_FG_49 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_fga = 0
    s_fgm = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_fga += float(game_data['Home FGA (40-49)'][j])
            s_fgm += float(game_data['Home FGM (40-49)'][j])
        else:
            s_fga += float(game_data['Away FGA (40-49)'][j])
            s_fgm += float(game_data['Away FGM (40-49)'][j])
    
    if(s_fga==0):
        home_FG_49[i] = 0.0
    else: 
        home_FG_49[i] = round(s_fgm / s_fga, 4)

# home team field goal success rate from 50+ yards
home_FG_50 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_fga = 0
    s_fgm = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j][0]==team):
            s_fga += float(game_data['Home FGA (50+)'][j])
            s_fgm += float(game_data['Home FGM (50+)'][j])
        else:
            s_fga += float(game_data['Away FGA (50+)'][j])
            s_fgm += float(game_data['Away FGM (50+)'][j])
    
    if(s_fga==0):
        home_FG_50[i] = 0.0
    else: 
        home_FG_50[i] = round(s_fgm / s_fga, 4)
        
# home team yards per punt
home_yds_per_punt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_punts = 0
    s_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_punts += float(game_data['Home Punts'][j])
            s_yds += float(game_data['Home Punting Yds'][j])
        else:
            s_punts += float(game_data['Away Punts'][j])
            s_yds += float(game_data['Away Punting Yds'][j])
    
    if(s_punts==0):
        home_yds_per_punt[i] = 0.0
    else:
        home_yds_per_punt[i] = round(s_yds / s_punts, 2)

# home team % punts inside opponent 20-yard line
home_punts_in_20 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_punts = 0
    s_in_20 = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_punts += float(game_data['Home Punts'][j])
            s_in_20 += float(game_data['Home Punts Inside 20'][j])
        else:
            s_punts += float(game_data['Away Punts'][j])
            s_in_20 += float(game_data['Away Punts Inside 20'][j])
    
    if(s_punts==0):
        home_punts_in_20[i] = 0.0
    else:
        home_punts_in_20[i] = round(s_in_20 / s_punts, 4)

# home team offensive penalty yards per game
home_off_pen_yds = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += float(game_data['Home Offense Penalty Yds'][j])
        else:
            s += float(game_data['Away Offense Penalty Yds'][j])
            
    home_off_pen_yds[i] = round(s / n_game_stats, 2)

# home team defensive penalty yards per game
home_def_pen_yds = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += float(game_data['Home Defense Penalty Yds'][j])
        else:
            s += float(game_data['Away Defense Penalty Yds'][j])
            
    home_def_pen_yds[i] = round(s / n_game_stats, 2)

# home team rush yards per carry
home_off_rush_ypc = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Net Rush Yds'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Net Rush Yds'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (RE)
home_off_rush_ypc_re = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_re[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (RT)
home_off_rush_ypc_rt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_rt[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (RG)
home_off_rush_ypc_rg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_rg[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (C)
home_off_rush_ypc_c = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - C'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - C'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_c[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (LG)
home_off_rush_ypc_lg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_lg[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (LT)
home_off_rush_ypc_lt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_lt[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry (LE)
home_off_rush_ypc_le = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_off_rush_ypc_le[i] = round(s_yds / s_carries, 3)

# home team rush yards per carry allowed
home_def_rush_ypc = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Net Rush Yds'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Net Rush Yds'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (RE)
home_def_rush_ypc_re = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_re[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (RT)
home_def_rush_ypc_rt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_rt[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (RG)
home_def_rush_ypc_rg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_rg[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (C)
home_def_rush_ypc_c = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - C'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - C'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_c[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (LG)
home_def_rush_ypc_lg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_lg[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (LT)
home_def_rush_ypc_lt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_lt[i] = round(s_yds / s_carries, 3)
    
# home team rush yards per carry allowed (LE)
home_def_rush_ypc_le = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    home_def_rush_ypc_le[i] = round(s_yds / s_carries, 3)

# home team adjusted net yards per passing attempt: 
# (gross pass yds + 20*(pass TDs) - 45*(ints thrown) - sack yds taken) / (passing attempts + sacks taken)
home_off_any_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_tds = 0
    s_ints = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_tds += float(game_data['Home Pass TDs'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_tds += float(game_data['Away Pass TDs'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    home_off_any_a[i] = round((s_pass_yds + 20*s_tds - 45*s_ints - s_sack_yds) / (s_pass_att + s_sacks), 3)
    
# home team net yards per passing attempt: 
# (gross pass yds - sack yds taken) / (passing attempts + sacks taken)
home_off_ny_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    home_off_ny_a[i] = round((s_pass_yds - s_sack_yds) / (s_pass_att + s_sacks), 3)


# home team adjusted net yards per passing attempt allowed: 
# (gross pass yds + 20*(pass TDs) - 45*(ints thrown) - sack yds taken) / (passing attempts + sacks taken)
home_def_any_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_tds = 0
    s_ints = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_tds += float(game_data['Home Pass TDs'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_tds += float(game_data['Away Pass TDs'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    home_def_any_a[i] = round((s_pass_yds + 20*s_tds - 45*s_ints - s_sack_yds) / (s_pass_att + s_sacks), 3)
    
# home team net yards per passing attempt allowed: 
# (gross pass yds - sack yds taken) / (passing attempts + sacks taken)
home_def_ny_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    home_def_ny_a[i] = round((s_pass_yds - s_sack_yds) / (s_pass_att + s_sacks), 3)
    
# home team explosive play rate:
# (20+ yard pass plays + 10+ yard rush plays) / (passing attempts + sacks taken + rush plays)
home_off_epr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_expl = 0
    s_plays = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_expl += float(game_data['Home Explosive Plays'][j])
            s_plays += float(game_data['Home Rush Plays'][j]) + float(game_data['Home Pass Attempts'][j]) 
            + float(game_data['Home Sacks Taken'][j]) 
        else:
            s_expl += float(game_data['Away Explosive Plays'][j])
            s_plays += float(game_data['Away Rush Plays'][j]) + float(game_data['Away Pass Attempts'][j]) 
            + float(game_data['Away Sacks Taken'][j]) 
    
    home_off_epr[i] = round(s_expl / s_plays, 4)

# home team explosive play rate allowed:
# (20+ yard pass plays + 10+ yard rush plays) / (passing attempts + sacks taken + rush plays)
home_def_epr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_expl = 0
    s_plays = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_expl += float(game_data['Home Explosive Plays'][j])
            s_plays += float(game_data['Home Rush Plays'][j]) + float(game_data['Home Pass Attempts'][j])
            + float(game_data['Home Sacks Taken'][j]) 
        else:
            s_expl += float(game_data['Away Explosive Plays'][j])
            s_plays += float(game_data['Away Rush Plays'][j]) + float(game_data['Away Pass Attempts'][j]) 
            + float(game_data['Away Sacks Taken'][j]) 
    
    home_def_epr[i] = round(s_expl / s_plays, 4)
    
# home team early-down pass success rate
home_off_edps = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Pass Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Pass Successes'][j])
    
    home_off_edps[i] = round(s_ed_s / s_ed_a, 4)

# home team early-down pass success rate allowed
home_def_edps = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Pass Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Pass Successes'][j])
    
    home_def_edps[i] = round(s_ed_s / s_ed_a, 4)

# home team early-down rush success rate
home_off_edrs = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Rush Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Rush Successes'][j])
    
    home_off_edrs[i] = round(s_ed_s / s_ed_a, 4)

# home team early-down rush success rate allowed
home_def_edrs = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Rush Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Rush Successes'][j])
    
    home_def_edrs[i] = round(s_ed_s / s_ed_a, 4)

# home team fumbles lost (last n_game_stats games)
home_fumbles_lost = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += float(game_data['Home Fumbles Lost'][j])
        else:
            s += float(game_data['Away Fumbles Lost'][j])
    
    home_fumbles_lost[i] = s

# home team fumbles taken (last n_game_stats games)
home_fumbles_taken = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s += float(game_data['Home Fumbles Lost'][j])
        else:
            s += float(game_data['Away Fumbles Lost'][j])
    
    home_fumbles_taken[i] = s

# home team interceptions thrown rate
home_off_int_rate = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_a = 0
    s_ints = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_a += float(game_data['Home Pass Attempts'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
        else:
            s_a += float(game_data['Away Pass Attempts'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
    
    home_off_int_rate[i] = round(s_ints / s_a, 4)

# home team interceptions caught rate
home_def_int_rate = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_a = 0
    s_ints = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_a += float(game_data['Home Pass Attempts'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
        else:
            s_a += float(game_data['Away Pass Attempts'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
    
    home_def_int_rate[i] = round(s_ints / s_a, 4)
    
# home team yards per punt return
home_off_yppr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Punt Returns'][j])
            s_yds += float(game_data['Home Punt Return Yds'][j])
        else:
            s_returns += float(game_data['Away Punt Returns'][j])
            s_yds += float(game_data['Away Punt Return Yds'][j])
            
    if(s_returns==0):
        home_off_yppr[i] = 0.0
    else:
        home_off_yppr[i] = round(s_yds / s_returns, 2)

# home team yards per punt return allowed
home_def_yppr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Punt Returns'][j])
            s_yds += float(game_data['Home Punt Return Yds'][j])
        else:
            s_returns += float(game_data['Away Punt Returns'][j])
            s_yds += float(game_data['Away Punt Return Yds'][j])
    
    if(s_returns==0):
        home_def_yppr[i] = 0.0
    else:
        home_def_yppr[i] = round(s_yds / s_returns, 2)
    
# home team yards per kickoff return
home_off_ypkr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Kickoff Returns'][j])
            s_yds += float(game_data['Home Kickoff Return Yds'][j])
        else:
            s_returns += float(game_data['Away Kickoff Returns'][j])
            s_yds += float(game_data['Away Kickoff Return Yds'][j])
    if(s_returns==0):
        home_off_ypkr[i] = 0.0
    else:
        home_off_ypkr[i] = round(s_yds / s_returns, 2)

# home team yards per kickoff return allowed
home_def_ypkr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Kickoff Returns'][j])
            s_yds += float(game_data['Home Kickoff Return Yds'][j])
        else:
            s_returns += float(game_data['Away Kickoff Returns'][j])
            s_yds += float(game_data['Away Kickoff Return Yds'][j])
    
    if(s_returns==0):
        home_def_ypkr[i] = 0.0
    else:
        home_def_ypkr[i] = round(s_yds / s_returns, 2)

# home team average offensive starting field position
home_off_avg_sfp = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += game_data['Home Avg Starting Field Position'][j]
        else:
            s += game_data['Away Avg Starting Field Position'][j]
    
    home_off_avg_sfp[i] = round(s / n_game_stats, 2)

# home team average defensive starting field position
home_def_avg_sfp = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s += game_data['Home Avg Starting Field Position'][j]
        else:
            s += game_data['Away Avg Starting Field Position'][j]
    
    home_def_avg_sfp[i] = round(s / n_game_stats, 2)
    
# home team short pass completion rate
home_off_spcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Short Pass Attempts'][j])
            s_comp += float(game_data['Home Short Pass Completions'][j])
        else:
            s_att += float(game_data['Away Short Pass Attempts'][j])
            s_comp += float(game_data['Away Short Pass Completions'][j])
    
    home_off_spcr[i] = round(s_comp / s_att, 4)

# home team short pass completion rate allowed
home_def_spcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Short Pass Attempts'][j])
            s_comp += float(game_data['Home Short Pass Completions'][j])
        else:
            s_att += float(game_data['Away Short Pass Attempts'][j])
            s_comp += float(game_data['Away Short Pass Completions'][j])
    
    home_def_spcr[i] = round(s_comp / s_att, 4)
    
# home team deep pass completion rate
home_off_dpcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Deep Pass Attempts'][j])
            s_comp += float(game_data['Home Deep Pass Completions'][j])
        else:
            s_att += float(game_data['Away Deep Pass Attempts'][j])
            s_comp += float(game_data['Away Deep Pass Completions'][j])
    
    home_off_dpcr[i] = round(s_comp / s_att, 4)

# home team deep pass completion rate allowed
home_def_dpcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Deep Pass Attempts'][j])
            s_comp += float(game_data['Home Deep Pass Completions'][j])
        else:
            s_att += float(game_data['Away Deep Pass Attempts'][j])
            s_comp += float(game_data['Away Deep Pass Completions'][j])
    
    home_def_dpcr[i] = round(s_comp / s_att, 4)

# home team third down conversion rate
home_off_tdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Third Down Attempts'][j])
            s_conv += float(game_data['Home Third Down Conversions'][j])
        else:
            s_att += float(game_data['Away Third Down Attempts'][j])
            s_conv += float(game_data['Away Third Down Conversions'][j])
    
    home_off_tdcr[i] = round(s_conv / s_att, 4)

# home team third down conversion rate allowed
home_def_tdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Third Down Attempts'][j])
            s_conv += float(game_data['Home Third Down Conversions'][j])
        else:
            s_att += float(game_data['Away Third Down Attempts'][j])
            s_conv += float(game_data['Away Third Down Conversions'][j])
    
    home_def_tdcr[i] = round(s_conv / s_att, 4)

# home team fourth down conversion rate
home_off_fdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Fourth Down Attempts'][j])
            s_conv += float(game_data['Home Fourth Down Conversions'][j])
        else:
            s_att += float(game_data['Away Fourth Down Attempts'][j])
            s_conv += float(game_data['Away Fourth Down Conversions'][j])
    
    if(s_att==0):
        home_off_fdcr[i] = 0.0
    else:
        home_off_fdcr[i] = round(s_conv / s_att, 4)

# home team fourth down conversion rate allowed
home_def_fdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Fourth Down Attempts'][j])
            s_conv += float(game_data['Home Fourth Down Conversions'][j])
        else:
            s_att += float(game_data['Away Fourth Down Attempts'][j])
            s_conv += float(game_data['Away Fourth Down Conversions'][j])
    
    if(s_att==0):
        home_def_fdcr[i] = 0.0
    else:
        home_def_fdcr[i] = round(s_conv / s_att, 4)

# home team red zone efficiency
home_off_rze = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_trips = 0
    s_tds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_trips += float(game_data['Home Trips To Red Zone'][j])
            s_tds += float(game_data['Home Red Zone TDs'][j])
        else:
            s_trips += float(game_data['Away Trips To Red Zone'][j])
            s_tds += float(game_data['Away Red Zone TDs'][j])
    
    if(s_trips==0):
        home_off_rze[i] = 0.0
    else:
        home_off_rze[i] = round(s_tds / s_trips, 4)

# home team red zone efficiency allowed
home_def_rze = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_trips = 0
    s_tds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_trips += float(game_data['Home Trips To Red Zone'][j])
            s_tds += float(game_data['Home Red Zone TDs'][j])
        else:
            s_trips += float(game_data['Away Trips To Red Zone'][j])
            s_tds += float(game_data['Away Red Zone TDs'][j])
    
    if(s_trips==0):
        home_def_rze[i] = 0.0
    else:
        home_def_rze[i] = round(s_tds / s_trips, 4)

# home team points allowed per 100 yards
home_def_pts_100 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds_allowed = 0
    s_pts_allowed = 0
    
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds_allowed += float(game_data['Home Net Rush Yds'][j]) + float(game_data['Home Gross Pass Yds'][j])
            - float(game_data['Home Sack Yds Taken'][j])
            s_pts_allowed += float(game_data['Home Pts'][j])
        else:
            s_yds_allowed += float(game_data['Away Net Rush Yds'][j]) + float(game_data['Away Gross Pass Yds'][j])
            - float(game_data['Away Sack Yds Taken'][j])
            s_pts_allowed += float(game_data['Away Pts'][j])
    
    home_def_pts_100[i] = round(s_pts_allowed / (s_yds_allowed / 100), 3)
    
# home team turnover margin last week
#(Other Team Interceptions Thrown + Other Team Fumbles Lost) - (This Team Interceptions Thrown + This Team Away Fumbles Lost)
home_lw_tom = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    previous_games = game_data[:start_game_index + i]
    last_away_game_i = previous_games.index[previous_games['Away Team']==team].tolist()[-1]
    last_home_game_i = previous_games.index[previous_games['Home Team']==team].tolist()[-1]
    other_int_thrown = 0
    other_fum_lost = 0
    this_int_thrown = 0
    this_fum_lost = 0
    if(last_home_game_i > last_away_game_i):
        other_int_thrown = previous_games['Away Interceptions Thrown'].to_numpy()[last_home_game_i]
        other_fum_lost = previous_games['Away Fumbles Lost'].to_numpy()[last_home_game_i]
        this_int_thrown = previous_games['Home Interceptions Thrown'].to_numpy()[last_home_game_i]
        this_fum_lost = previous_games['Home Fumbles Lost'].to_numpy()[last_home_game_i]
    else:
        other_int_thrown = previous_games['Home Interceptions Thrown'].to_numpy()[last_away_game_i]
        other_fum_lost = previous_games['Home Fumbles Lost'].to_numpy()[last_away_game_i]
        this_int_thrown = previous_games['Away Interceptions Thrown'].to_numpy()[last_away_game_i]
        this_fum_lost = previous_games['Away Fumbles Lost'].to_numpy()[last_away_game_i]
    
    home_lw_tom[i] = (other_int_thrown + other_fum_lost) - (this_int_thrown + this_fum_lost )

# home team season over-under record margin (number of games hitting over - number of games hitting under)
home_margin_o_u = np.zeros((num_total_games,1))
ids = game_data['GameId'].to_numpy()
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    previous_games = game_data[:start_game_index + i]
    
    month = int(str(ids[start_game_index + i])[4:6])
    year =  int(str(ids[start_game_index + i])[:4])

    fst_game = 0
    if(month > 6):
        fst_game = ids[ids > (int(str(year) + '07') * 10000)][0]
    else:
        fst_game = ids[ids > (int(str(year - 1) + '07') * 10000)][0]
    
    season_games = pd.DataFrame()
    season_home_games = pd.DataFrame()
    season_away_games = pd.DataFrame()
    if(fst_game!=gameId):
        fst_game_i = np.where(ids==fst_game)[0][0]
        season_games = previous_games[fst_game_i:]
        season_home_games = season_games[season_games['Home Team']==team]
        season_away_games = season_games[season_games['Away Team']==team]
        
    season_games = pd.concat([season_home_games, season_away_games])
    
    games_hit_o = 0
    games_hit_u = 0

    for j in range(len(season_games)):
        if((season_games['Home Pts'].to_numpy()[j] + season_games['Away Pts'].to_numpy()[j]) 
           > season_games['Vegas Over Under'].to_numpy()[j]):
            games_hit_o += 1
        elif((season_games['Home Pts'].to_numpy()[j] + season_games['Away Pts'].to_numpy()[j]) 
             < season_games['Vegas Over Under'].to_numpy()[j]):
            games_hit_u += 1

    home_margin_o_u[i] = games_hit_o - games_hit_u

# home team season record ATS margin (number of games won ATS - number of games lost ATS)
home_margin_ats = np.zeros((num_total_games,1))
ids = game_data['GameId'].to_numpy()
for i in range(num_total_games):
    team = avail_game_data['Home Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    previous_games = game_data[:start_game_index + i]
    
    month = int(str(ids[start_game_index + i])[4:6])
    year =  int(str(ids[start_game_index + i])[:4])

    fst_game = 0
    if(month > 6):
        fst_game = ids[ids > (int(str(year) + '07') * 10000)][0]
    else:
        fst_game = ids[ids > (int(str(year - 1) + '07') * 10000)][0]
    
    season_games = pd.DataFrame()
    season_home_games = pd.DataFrame()
    season_away_games = pd.DataFrame()
    if(fst_game!=gameId):
        fst_game_i = np.where(ids==fst_game)[0][0]
        season_games = previous_games[fst_game_i:]
        season_home_games = season_games[season_games['Home Team']==team]
        season_away_games = season_games[season_games['Away Team']==team]
        
    season_games = pd.concat([season_home_games, season_away_games])
    
    games_won_ats = 0
    games_lost_ats = 0

    for j in range(len(season_games)):
        if(season_games['Vegas Spread Favorite'].to_numpy()[j]==team):
            if(season_games['Home Team'].to_numpy()[j]==team):
                if((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j])
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
            else:
                if((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
        else:
            if(season_games['Home Team'].to_numpy()[j]==team):
                if((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
            else:
                if((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
    
    home_margin_ats[i] = games_won_ats - games_lost_ats

In [20]:
# create away team features

# one-hot encoded: for each feature, 1 if away team has a specific head coach, 0 otherwise
away_coach_text = avail_game_data[['Away Coach']].to_numpy()
away_coach = enc.fit(away_coach_text).transform(away_coach_text).A

# number of days since away team's last game
away_rest_days = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    game_date = datetime.strptime(avail_game_data['Game Date'].to_numpy()[i],'%m/%d/%Y')
    previous_games = game_data[:start_game_index + i]
    last_away_game_i = previous_games.index[previous_games['Away Team']==team].tolist()[-1]
    last_home_game_i = previous_games.index[previous_games['Home Team']==team].tolist()[-1]
    last_game_i = max(last_away_game_i, last_home_game_i)
    last_game_date = datetime.strptime(game_data['Game Date'].to_numpy()[last_game_i],'%m/%d/%Y')
    away_rest_days[i] = (game_date - last_game_date).days

# away team average time of possession (not including overtime)
away_avg_pos_time = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += game_data['Home Time of Posession'][j]
        else:
            s += game_data['Away Time of Posession'][j]
    
    away_avg_pos_time[i] = round(s / n_game_stats, 2)

# away team PAT success rate
away_PAT = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_PAT_a = 0
    s_PAT_m = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_PAT_a += float(game_data['Home PAT Attempts'][j])
            s_PAT_m += float(game_data['Home PAT Made'][j])
        else:
            s_PAT_a += float(game_data['Away PAT Attempts'][j])
            s_PAT_m += float(game_data['Away PAT Made'][j])
    
    if(s_PAT_a==0):
        away_PAT[i] = 0.0
    else:
        away_PAT[i] = round(s_PAT_m / s_PAT_a, 4)

# away team field goal success rate from 0-39 yards
away_FG_39 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_fga = 0
    s_fgm = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j][0]==team):
            s_fga += float(game_data['Home FGA (0-39)'][j])
            s_fgm += float(game_data['Home FGM (0-39)'][j])
        else:
            s_fga += float(game_data['Away FGA (0-39)'][j])
            s_fgm += float(game_data['Away FGM (0-39)'][j])
    
    if(s_fga==0):
        away_FG_39[i] = 0.0
    else: 
        away_FG_39[i] = round(s_fgm / s_fga, 4)
        
# away team field goal success rate from 40-49 yards
away_FG_49 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_fga = 0
    s_fgm = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_fga += float(game_data['Home FGA (40-49)'][j])
            s_fgm += float(game_data['Home FGM (40-49)'][j])
        else:
            s_fga += float(game_data['Away FGA (40-49)'][j])
            s_fgm += float(game_data['Away FGM (40-49)'][j])
    
    if(s_fga==0):
        away_FG_49[i] = 0.0
    else: 
        away_FG_49[i] = round(s_fgm / s_fga, 4)

# away team field goal success rate from 50+ yards
away_FG_50 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_fga = 0
    s_fgm = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j][0]==team):
            s_fga += float(game_data['Home FGA (50+)'][j])
            s_fgm += float(game_data['Home FGM (50+)'][j])
        else:
            s_fga += float(game_data['Away FGA (50+)'][j])
            s_fgm += float(game_data['Away FGM (50+)'][j])
    
    if(s_fga==0):
        away_FG_50[i] = 0.0
    else: 
        away_FG_50[i] = round(s_fgm / s_fga, 4)

# away team yards per punt
away_yds_per_punt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_punts = 0
    s_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_punts += float(game_data['Home Punts'][j])
            s_yds += float(game_data['Home Punting Yds'][j])
        else:
            s_punts += float(game_data['Away Punts'][j])
            s_yds += float(game_data['Away Punting Yds'][j])
    
    if(s_punts==0):
        away_yds_per_punt[i] = 0.0
    else:
        away_yds_per_punt[i] = round(s_yds / s_punts, 2)

# away team % punts inside opponent 20-yard line
away_punts_in_20 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_punts = 0
    s_in_20 = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_punts += float(game_data['Home Punts'][j])
            s_in_20 += float(game_data['Home Punts Inside 20'][j])
        else:
            s_punts += float(game_data['Away Punts'][j])
            s_in_20 += float(game_data['Away Punts Inside 20'][j])
    
    if(s_punts==0):
        away_punts_in_20[i] = 0.0
    else:
        away_punts_in_20[i] = round(s_in_20 / s_punts, 4)

# away team offensive penalty yards per game
away_off_pen_yds = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += float(game_data['Home Offense Penalty Yds'][j])
        else:
            s += float(game_data['Away Offense Penalty Yds'][j])
            
    away_off_pen_yds[i] = round(s / n_game_stats, 2)

# away team defensive penalty yards per game
away_def_pen_yds = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += float(game_data['Home Defense Penalty Yds'][j])
        else:
            s += float(game_data['Away Defense Penalty Yds'][j])
            
    away_def_pen_yds[i] = round(s / n_game_stats, 2)

# away team rush yards per carry
away_off_rush_ypc = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Net Rush Yds'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Net Rush Yds'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (RE)
away_off_rush_ypc_re = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_re[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (RT)
away_off_rush_ypc_rt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_rt[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (RG)
away_off_rush_ypc_rg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_rg[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (C)
away_off_rush_ypc_c = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - C'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - C'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_c[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (LG)
away_off_rush_ypc_lg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_lg[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (LT)
away_off_rush_ypc_lt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_lt[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry (LE)
away_off_rush_ypc_le = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_off_rush_ypc_le[i] = round(s_yds / s_carries, 3)

# away team rush yards per carry allowed
away_def_rush_ypc = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Net Rush Yds'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Net Rush Yds'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry allowed (RE)
away_def_rush_ypc_re = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_re[i] = round(s_yds / s_carries, 3)

# away team rush yards per carry allowed (RT)
away_def_rush_ypc_rt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_rt[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry allowed (RG)
away_def_rush_ypc_rg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - RG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - RG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_rg[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry allowed (C)
away_def_rush_ypc_c = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - C'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - C'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_c[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry allowed (LG)
away_def_rush_ypc_lg = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LG'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LG'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_lg[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry allowed (LT)
away_def_rush_ypc_lt = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LT'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LT'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_lt[i] = round(s_yds / s_carries, 3)
    
# away team rush yards per carry allowed (LE)
away_def_rush_ypc_le = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds = 0
    s_carries = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds += float(game_data['Home Rush Yds - LE'][j])
            s_carries += float(game_data['Home Rush Plays'][j])
        else:
            s_yds += float(game_data['Away Rush Yds - LE'][j])
            s_carries += float(game_data['Away Rush Plays'][j])
    
    away_def_rush_ypc_le[i] = round(s_yds / s_carries, 3)
    
# away team adjusted net yards per passing attempt: 
# (gross pass yds + 20*(pass TDs) - 45*(ints thrown) - sack yds taken) / (passing attempts + sacks taken)
away_off_any_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_tds = 0
    s_ints = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_tds += float(game_data['Home Pass TDs'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_tds += float(game_data['Away Pass TDs'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    away_off_any_a[i] = round((s_pass_yds + 20*s_tds - 45*s_ints - s_sack_yds) / (s_pass_att + s_sacks), 3)
    
# away team net yards per passing attempt: 
# (gross pass yds - sack yds taken) / (passing attempts + sacks taken)
away_off_ny_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    away_off_ny_a[i] = round((s_pass_yds - s_sack_yds) / (s_pass_att + s_sacks), 3)

# away team adjusted net yards per passing attempt allowed:
# (gross pass yds + 20*(pass TDs) - 45*(ints thrown) - sack yds taken) / (passing attempts + sacks taken)
away_def_any_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_tds = 0
    s_ints = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_tds += float(game_data['Home Pass TDs'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_tds += float(game_data['Away Pass TDs'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    away_def_any_a[i] = round((s_pass_yds + 20*s_tds - 45*s_ints - s_sack_yds) / (s_pass_att + s_sacks), 3)
    
# away team net yards per passing attempt allowed:
# (gross pass yds - sack yds taken) / (passing attempts + sacks taken)
away_def_ny_a = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_pass_yds = 0
    s_pass_att = 0
    s_sacks = 0
    s_sack_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_pass_yds += float(game_data['Home Gross Pass Yds'][j])
            s_pass_att += float(game_data['Home Pass Attempts'][j])
            s_sacks += float(game_data['Home Sacks Taken'][j])
            s_sack_yds += float(game_data['Home Sack Yds Taken'][j])
        else:
            s_pass_yds += float(game_data['Away Gross Pass Yds'][j])
            s_pass_att += float(game_data['Away Pass Attempts'][j])
            s_sacks += float(game_data['Away Sacks Taken'][j])
            s_sack_yds += float(game_data['Away Sack Yds Taken'][j])
    
    away_def_ny_a[i] = round((s_pass_yds - s_sack_yds) / (s_pass_att + s_sacks), 3)
    
# away team explosive play rate:
# (20+ yard pass plays + 10+ yard rush plays) / (passing attempts + sacks taken + rush plays)
away_off_epr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_expl = 0
    s_plays = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_expl += float(game_data['Home Explosive Plays'][j])
            s_plays += float(game_data['Home Rush Plays'][j]) + float(game_data['Home Pass Attempts'][j]) 
            + float(game_data['Home Sacks Taken'][j]) 
        else:
            s_expl += float(game_data['Away Explosive Plays'][j])
            s_plays += float(game_data['Away Rush Plays'][j]) + float(game_data['Away Pass Attempts'][j]) 
            + float(game_data['Away Sacks Taken'][j]) 
    
    away_off_epr[i] = round(s_expl / s_plays, 4)

# away team explosive play rate allowed:
# (20+ yard pass plays + 10+ yard rush plays) / (passing attempts + sacks taken + rush plays)
away_def_epr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_expl = 0
    s_plays = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_expl += float(game_data['Home Explosive Plays'][j])
            s_plays += float(game_data['Home Rush Plays'][j]) + float(game_data['Home Pass Attempts'][j])
            + float(game_data['Home Sacks Taken'][j]) 
        else:
            s_expl += float(game_data['Away Explosive Plays'][j])
            s_plays += float(game_data['Away Rush Plays'][j]) + float(game_data['Away Pass Attempts'][j]) 
            + float(game_data['Away Sacks Taken'][j]) 
    
    away_def_epr[i] = round(s_expl / s_plays, 4)
    
# away team early-down pass success rate
away_off_edps = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Pass Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Pass Successes'][j])
    
    away_off_edps[i] = round(s_ed_s / s_ed_a, 4)

# away team early-down pass success rate allowed
away_def_edps = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Pass Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Pass Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Pass Successes'][j])
    
    away_def_edps[i] = round(s_ed_s / s_ed_a, 4)

# away team early-down rush success rate
away_off_edrs = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Rush Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Rush Successes'][j])
    
    away_off_edrs[i] = round(s_ed_s / s_ed_a, 4)

# away team early-down rush success rate allowed
away_def_edrs = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_ed_a = 0
    s_ed_s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_ed_a += float(game_data['Home Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Home Early-Down Rush Successes'][j])
        else:
            s_ed_a += float(game_data['Away Early-Down Rush Attempts'][j])
            s_ed_s += float(game_data['Away Early-Down Rush Successes'][j])
    
    away_def_edrs[i] = round(s_ed_s / s_ed_a, 4)

# away team fumbles lost (last n_game_stats games)
away_fumbles_lost = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += float(game_data['Home Fumbles Lost'][j])
        else:
            s += float(game_data['Away Fumbles Lost'][j])
    
    away_fumbles_lost[i] = s

# away team fumbles taken (last n_game_stats games)
away_fumbles_taken = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s += float(game_data['Home Fumbles Lost'][j])
        else:
            s += float(game_data['Away Fumbles Lost'][j])
    
    away_fumbles_taken[i] = s

# away team interceptions thrown rate
away_off_int_rate = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_a = 0
    s_ints = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_a += float(game_data['Home Pass Attempts'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
        else:
            s_a += float(game_data['Away Pass Attempts'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
    
    away_off_int_rate[i] = round(s_ints / s_a, 4)

# away team interceptions caught rate
away_def_int_rate = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_a = 0
    s_ints = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_a += float(game_data['Home Pass Attempts'][j])
            s_ints += float(game_data['Home Interceptions Thrown'][j])
        else:
            s_a += float(game_data['Away Pass Attempts'][j])
            s_ints += float(game_data['Away Interceptions Thrown'][j])
    
    away_def_int_rate[i] = round(s_ints / s_a, 4)
    
# away team yards per punt return
away_off_yppr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Punt Returns'][j])
            s_yds += float(game_data['Home Punt Return Yds'][j])
        else:
            s_returns += float(game_data['Away Punt Returns'][j])
            s_yds += float(game_data['Away Punt Return Yds'][j])
    if(s_returns==0):
        away_off_yppr[i] = 0.0
    else:
        away_off_yppr[i] = round(s_yds / s_returns, 2)

# away team yards per punt return allowed
away_def_yppr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Punt Returns'][j])
            s_yds += float(game_data['Home Punt Return Yds'][j])
        else:
            s_returns += float(game_data['Away Punt Returns'][j])
            s_yds += float(game_data['Away Punt Return Yds'][j])
    if(s_returns==0):
        away_def_yppr[i] = 0.0
    else:
        away_def_yppr[i] = round(s_yds / s_returns, 2)
    
# away team yards per kickoff return
away_off_ypkr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Kickoff Returns'][j])
            s_yds += float(game_data['Home Kickoff Return Yds'][j])
        else:
            s_returns += float(game_data['Away Kickoff Returns'][j])
            s_yds += float(game_data['Away Kickoff Return Yds'][j])
    if(s_returns==0):
        away_off_ypkr[i] = 0.0
    else:
        away_off_ypkr[i] = round(s_yds / s_returns, 2)

# away team yards per kickoff return allowed
away_def_ypkr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_returns = 0
    s_yds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_returns += float(game_data['Home Kickoff Returns'][j])
            s_yds += float(game_data['Home Kickoff Return Yds'][j])
        else:
            s_returns += float(game_data['Away Kickoff Returns'][j])
            s_yds += float(game_data['Away Kickoff Return Yds'][j])
    if(s_returns==0):
        away_def_ypkr[i] = 0.0
    else:
        away_def_ypkr[i] = round(s_yds / s_returns, 2)
    
# away team average offensive starting field position
away_off_avg_sfp = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s += game_data['Home Avg Starting Field Position'][j]
        else:
            s += game_data['Away Avg Starting Field Position'][j]
    
    away_off_avg_sfp[i] = round(s / n_game_stats, 2)

# away team average defensive starting field position
away_def_avg_sfp = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s += game_data['Home Avg Starting Field Position'][j]
        else:
            s += game_data['Away Avg Starting Field Position'][j]
    
    away_def_avg_sfp[i] = round(s / n_game_stats, 2)
    
# away team short pass completion rate
away_off_spcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Short Pass Attempts'][j])
            s_comp += float(game_data['Home Short Pass Completions'][j])
        else:
            s_att += float(game_data['Away Short Pass Attempts'][j])
            s_comp += float(game_data['Away Short Pass Completions'][j])
    
    away_off_spcr[i] = round(s_comp / s_att, 4)

# away team short pass completion rate allowed
away_def_spcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Short Pass Attempts'][j])
            s_comp += float(game_data['Home Short Pass Completions'][j])
        else:
            s_att += float(game_data['Away Short Pass Attempts'][j])
            s_comp += float(game_data['Away Short Pass Completions'][j])
    
    away_def_spcr[i] = round(s_comp / s_att, 4)
    
# away team deep pass completion rate
away_off_dpcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Deep Pass Attempts'][j])
            s_comp += float(game_data['Home Deep Pass Completions'][j])
        else:
            s_att += float(game_data['Away Deep Pass Attempts'][j])
            s_comp += float(game_data['Away Deep Pass Completions'][j])
    
    away_off_dpcr[i] = round(s_comp / s_att, 4)

# away team deep pass completion rate allowed
away_def_dpcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_comp = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Deep Pass Attempts'][j])
            s_comp += float(game_data['Home Deep Pass Completions'][j])
        else:
            s_att += float(game_data['Away Deep Pass Attempts'][j])
            s_comp += float(game_data['Away Deep Pass Completions'][j])
    
    away_def_dpcr[i] = round(s_comp / s_att, 4)
    
# away team third down conversion rate
away_off_tdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Third Down Attempts'][j])
            s_conv += float(game_data['Home Third Down Conversions'][j])
        else:
            s_att += float(game_data['Away Third Down Attempts'][j])
            s_conv += float(game_data['Away Third Down Conversions'][j])
    
    away_off_tdcr[i] = round(s_conv / s_att, 4)

# away team third down conversion rate allowed
away_def_tdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Third Down Attempts'][j])
            s_conv += float(game_data['Home Third Down Conversions'][j])
        else:
            s_att += float(game_data['Away Third Down Attempts'][j])
            s_conv += float(game_data['Away Third Down Conversions'][j])
    
    away_def_tdcr[i] = round(s_conv / s_att, 4)

# away team fourth down conversion rate
away_off_fdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Fourth Down Attempts'][j])
            s_conv += float(game_data['Home Fourth Down Conversions'][j])
        else:
            s_att += float(game_data['Away Fourth Down Attempts'][j])
            s_conv += float(game_data['Away Fourth Down Conversions'][j])
    
    if(s_att==0):
        away_off_fdcr[i] = 0.0
    else:
        away_off_fdcr[i] = round(s_conv / s_att, 4)

# away team fourth down conversion rate allowed
away_def_fdcr = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_att = 0
    s_conv = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_att += float(game_data['Home Fourth Down Attempts'][j])
            s_conv += float(game_data['Home Fourth Down Conversions'][j])
        else:
            s_att += float(game_data['Away Fourth Down Attempts'][j])
            s_conv += float(game_data['Away Fourth Down Conversions'][j])
    
    if(s_att==0):
        away_def_fdcr[i] = 0.0
    else:
        away_def_fdcr[i] = round(s_conv / s_att, 4)

# away team red zone efficiency
away_off_rze = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_trips = 0
    s_tds = 0
    for j in games:
        if(game_data['Home Team'].to_numpy()[j]==team):
            s_trips += float(game_data['Home Trips To Red Zone'][j])
            s_tds += float(game_data['Home Red Zone TDs'][j])
        else:
            s_trips += float(game_data['Away Trips To Red Zone'][j])
            s_tds += float(game_data['Away Red Zone TDs'][j])
    
    if(s_trips==0):
        away_off_rze[i] = 0.0
    else:
        away_off_rze[i] = round(s_tds / s_trips, 4)

# away team red zone efficiency allowed
away_def_rze = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_trips = 0
    s_tds = 0
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_trips += float(game_data['Home Trips To Red Zone'][j])
            s_tds += float(game_data['Home Red Zone TDs'][j])
        else:
            s_trips += float(game_data['Away Trips To Red Zone'][j])
            s_tds += float(game_data['Away Red Zone TDs'][j])
    
    if(s_trips==0):
        away_def_rze[i] = 0.0
    else:
        away_def_rze[i] = round(s_tds / s_trips, 4)

# away team points allowed per 100 yards
away_def_pts_100 = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    games = get_previous_games_i(team, gameId)
    s_yds_allowed = 0
    s_pts_allowed = 0
    
    for j in games:
        if(game_data['Away Team'].to_numpy()[j]==team):
            s_yds_allowed += float(game_data['Home Net Rush Yds'][j]) + float(game_data['Home Gross Pass Yds'][j])
            - float(game_data['Home Sack Yds Taken'][j])
            s_pts_allowed += float(game_data['Home Pts'][j])
        else:
            s_yds_allowed += float(game_data['Away Net Rush Yds'][j]) + float(game_data['Away Gross Pass Yds'][j])
            - float(game_data['Away Sack Yds Taken'][j])
            s_pts_allowed += float(game_data['Away Pts'][j])
    
    away_def_pts_100[i] = round(s_pts_allowed / (s_yds_allowed / 100), 3)
    
# away team turnover margin last week
#(Other Team Interceptions Thrown + Other Team Fumbles Lost) - (This Team Interceptions Thrown + This Team Away Fumbles Lost)
away_lw_tom = np.zeros((num_total_games,1))
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    previous_games = game_data[:start_game_index + i]
    last_away_game_i = previous_games.index[previous_games['Away Team']==team].tolist()[-1]
    last_home_game_i = previous_games.index[previous_games['Home Team']==team].tolist()[-1]
    other_int_thrown = 0
    other_fum_lost = 0
    this_int_thrown = 0
    this_fum_lost = 0
    if(last_home_game_i > last_away_game_i):
        other_int_thrown = previous_games['Away Interceptions Thrown'].to_numpy()[last_home_game_i]
        other_fum_lost = previous_games['Away Fumbles Lost'].to_numpy()[last_home_game_i]
        this_int_thrown = previous_games['Home Interceptions Thrown'].to_numpy()[last_home_game_i]
        this_fum_lost = previous_games['Home Fumbles Lost'].to_numpy()[last_home_game_i]
    else:
        other_int_thrown = previous_games['Home Interceptions Thrown'].to_numpy()[last_away_game_i]
        other_fum_lost = previous_games['Home Fumbles Lost'].to_numpy()[last_away_game_i]
        this_int_thrown = previous_games['Away Interceptions Thrown'].to_numpy()[last_away_game_i]
        this_fum_lost = previous_games['Away Fumbles Lost'].to_numpy()[last_away_game_i]
    
    away_lw_tom[i] = (other_int_thrown + other_fum_lost) - (this_int_thrown + this_fum_lost )

# away team season over-under record margin (number of games hitting over - number of games hitting under)
away_margin_o_u = np.zeros((num_total_games,1))
ids = game_data['GameId'].to_numpy()
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    previous_games = game_data[:start_game_index + i]
    
    month = int(str(ids[start_game_index + i])[4:6])
    year =  int(str(ids[start_game_index + i])[:4])

    fst_game = 0
    if(month > 6):
        fst_game = ids[ids > (int(str(year) + '07') * 10000)][0]
    else:
        fst_game = ids[ids > (int(str(year - 1) + '07') * 10000)][0]
    
    season_games = pd.DataFrame()
    season_home_games = pd.DataFrame()
    season_away_games = pd.DataFrame()
    if(fst_game!=gameId):
        fst_game_i = np.where(ids==fst_game)[0][0]
        season_games = previous_games[fst_game_i:]
        season_home_games = season_games[season_games['Home Team']==team]
        season_away_games = season_games[season_games['Away Team']==team]
        
    season_games = pd.concat([season_home_games, season_away_games])
    
    games_hit_o = 0
    games_hit_u = 0

    for j in range(len(season_games)):
        if((season_games['Home Pts'].to_numpy()[j] + season_games['Away Pts'].to_numpy()[j]) 
           > season_games['Vegas Over Under'].to_numpy()[j]):
            games_hit_o += 1
        elif((season_games['Home Pts'].to_numpy()[j] + season_games['Away Pts'].to_numpy()[j]) 
             < season_games['Vegas Over Under'].to_numpy()[j]):
            games_hit_u += 1

    away_margin_o_u[i] = games_hit_o - games_hit_u

# away team season record ATS margin (number of games won ATS - number of games lost ATS)
away_margin_ats = np.zeros((num_total_games,1))
ids = game_data['GameId'].to_numpy()
for i in range(num_total_games):
    team = avail_game_data['Away Team'].to_numpy()[i]
    gameId = avail_game_data['GameId'].to_numpy()[i]
    previous_games = game_data[:start_game_index + i]
    
    month = int(str(ids[start_game_index + i])[4:6])
    year =  int(str(ids[start_game_index + i])[:4])

    fst_game = 0
    if(month > 6):
        fst_game = ids[ids > (int(str(year) + '07') * 10000)][0]
    else:
        fst_game = ids[ids > (int(str(year - 1) + '07') * 10000)][0]
    
    season_games = pd.DataFrame()
    season_home_games = pd.DataFrame()
    season_away_games = pd.DataFrame()
    if(fst_game!=gameId):
        fst_game_i = np.where(ids==fst_game)[0][0]
        season_games = previous_games[fst_game_i:]
        season_home_games = season_games[season_games['Home Team']==team]
        season_away_games = season_games[season_games['Away Team']==team]
        
    season_games = pd.concat([season_home_games, season_away_games])
    
    games_won_ats = 0
    games_lost_ats = 0

    for j in range(len(season_games)):
        if(season_games['Vegas Spread Favorite'].to_numpy()[j]==team):
            if(season_games['Home Team'].to_numpy()[j]==team):
                if((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
            else:
                if((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
        else:
            if(season_games['Home Team'].to_numpy()[j]==team):
                if((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Home Pts'].to_numpy()[j] - season_games['Away Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
            else:
                if((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   > season_games['Vegas Spread'].to_numpy()[j]):
                    games_won_ats += 1
                elif((season_games['Away Pts'].to_numpy()[j] - season_games['Home Pts'].to_numpy()[j]) 
                   < season_games['Vegas Spread'].to_numpy()[j]):
                    games_lost_ats += 1
    
    away_margin_ats[i] = games_won_ats - games_lost_ats

In [21]:
# create full data feature array
X = np.hstack((indoor_stadium, divisional, stadium, temp, humidity, wind_speed, home_coach, away_coach, home_rest_days, 
               away_rest_days, home_avg_pos_time, away_avg_pos_time, home_PAT, away_PAT, home_FG_39, away_FG_39, home_FG_49, 
               away_FG_49, home_FG_50, away_FG_50, home_yds_per_punt, away_yds_per_punt, home_punts_in_20, away_punts_in_20,
               home_off_pen_yds, away_off_pen_yds, home_def_pen_yds, away_def_pen_yds, home_off_rush_ypc, away_off_rush_ypc,
               home_def_rush_ypc, away_def_rush_ypc, home_off_rush_ypc_re, home_off_rush_ypc_rt, home_off_rush_ypc_rg, 
               home_off_rush_ypc_c, home_off_rush_ypc_lg, home_off_rush_ypc_lt, home_off_rush_ypc_le, home_def_rush_ypc_re, 
               home_def_rush_ypc_rt, home_def_rush_ypc_rg, home_def_rush_ypc_c, home_def_rush_ypc_lg, home_def_rush_ypc_lt, 
               home_def_rush_ypc_le, away_off_rush_ypc_re, away_off_rush_ypc_rt, away_off_rush_ypc_rg, away_off_rush_ypc_c, 
               away_off_rush_ypc_lg, away_off_rush_ypc_lt, away_off_rush_ypc_le, away_def_rush_ypc_re, away_def_rush_ypc_rt, 
               away_def_rush_ypc_rg, away_def_rush_ypc_c, away_def_rush_ypc_lg, away_def_rush_ypc_lt, away_def_rush_ypc_le, 
               home_off_any_a, away_off_any_a, home_def_any_a, away_def_any_a, home_off_ny_a, away_off_ny_a, home_def_ny_a, 
               away_def_ny_a, home_off_epr, away_off_epr, home_def_epr, away_def_epr, home_off_edps, away_off_edps, 
               home_def_edps, away_def_edps, home_off_edrs, away_off_edrs, home_def_edrs, away_def_edrs, home_fumbles_lost, 
               away_fumbles_lost, home_fumbles_taken, away_fumbles_taken, home_off_int_rate, away_off_int_rate, 
               home_def_int_rate, away_def_int_rate, home_off_yppr, away_off_yppr, home_def_yppr, away_def_yppr, home_off_ypkr, 
               away_off_ypkr, home_def_ypkr, away_def_ypkr, home_off_avg_sfp, away_off_avg_sfp, home_def_avg_sfp, 
               away_def_avg_sfp, home_off_spcr, away_off_spcr, home_def_spcr, away_def_spcr, home_off_dpcr, away_off_dpcr, 
               home_def_dpcr, away_def_dpcr, home_off_tdcr, away_off_tdcr, home_def_tdcr, away_def_tdcr, home_off_fdcr, 
               away_off_fdcr, home_def_fdcr, away_def_fdcr, home_off_rze, away_off_rze, home_def_rze, away_def_rze, 
               home_def_pts_100, away_def_pts_100, home_lw_tom, away_lw_tom, home_margin_o_u, away_margin_o_u, home_margin_ats, 
               away_margin_ats, point_total, vegas_over_under, hit_over, home_min_away_spread, vegas_home_min_away_spread, 
               home_covered_spread))

X.shape

(2008, 348)

In [22]:
# get names for columns in one-hot encoded stadium features 
stadium_col_names = enc.fit(stadium_text).categories_

# get names for columns in one-hot encoded home_coach features
home_coach_col_names = enc.fit(home_coach_text).categories_
for i in range(len(home_coach_col_names)):
    home_coach_col_names[i] = 'home_coach ' + home_coach_col_names[i]

# get names for columns in one-hot encoded away_coach features
away_coach_col_names = enc.fit(away_coach_text).categories_
for i in range(len(away_coach_col_names)):
    away_coach_col_names[i] = 'away_coach ' + away_coach_col_names[i]

In [23]:
# create list of column names for all features
col_names = []

list_1 = ['indoor_stadium', 'divisional']
list_2 = ['temp', 'humidity', 'wind_speed']
list_3 = ['home_rest_days', 'away_rest_days', 'home_avg_pos_time', 'away_avg_pos_time', 'home_PAT', 'away_PAT', 'home_FG_39',
          'away_FG_39', 'home_FG_49', 'away_FG_49', 'home_FG_50', 'away_FG_50', 'home_yds_per_punt','away_yds_per_punt',
          'home_punts_in_20', 'away_punts_in_20', 'home_off_pen_yds', 'away_off_pen_yds', 'home_def_pen_yds',
          'away_def_pen_yds', 'home_off_rush_ypc', 'away_off_rush_ypc', 'home_def_rush_ypc', 'away_def_rush_ypc', 
          'home_off_rush_ypc_re', 'home_off_rush_ypc_rt', 'home_off_rush_ypc_rg', 'home_off_rush_ypc_c', 'home_off_rush_ypc_lg',
          'home_off_rush_ypc_lt', 'home_off_rush_ypc_le', 'home_def_rush_ypc_re', 'home_def_rush_ypc_rt', 
          'home_def_rush_ypc_rg', 'home_def_rush_ypc_c', 'home_def_rush_ypc_lg', 'home_def_rush_ypc_lt', 'home_def_rush_ypc_le',
          'away_off_rush_ypc_re', 'away_off_rush_ypc_rt', 'away_off_rush_ypc_rg', 'away_off_rush_ypc_c', 'away_off_rush_ypc_lg',
          'away_off_rush_ypc_lt', 'away_off_rush_ypc_le', 'away_def_rush_ypc_re', 'away_def_rush_ypc_rt', 
          'away_def_rush_ypc_rg', 'away_def_rush_ypc_c', 'away_def_rush_ypc_lg', 'away_def_rush_ypc_lt', 'away_def_rush_ypc_le', 
          'home_off_any_a', 'away_off_any_a', 'home_def_any_a', 'away_def_any_a', 'home_off_ny_a', 'away_off_ny_a', 
          'home_def_ny_a', 'away_def_ny_a', 'home_off_epr', 'away_off_epr', 'home_def_epr', 'away_def_epr', 'home_off_edps',
          'away_off_edps', 'home_def_edps', 'away_def_edps', 'home_off_edrs', 'away_off_edrs', 'home_def_edrs', 'away_def_edrs',
          'home_fumbles_lost', 'away_fumbles_lost', 'home_fumbles_taken', 'away_fumbles_taken', 'home_off_int_rate',
          'away_off_int_rate', 'home_def_int_rate', 'away_def_int_rate', 'home_off_yppr', 'away_off_yppr', 'home_def_yppr',
          'away_def_yppr', 'home_off_ypkr', 'away_off_ypkr', 'home_def_ypkr', 'away_def_ypkr', 'home_off_avg_sfp', 
          'away_off_avg_sfp', 'home_def_avg_sfp', 'away_def_avg_sfp', 'home_off_spcr', 'away_off_spcr', 'home_def_spcr', 
          'away_def_spcr', 'home_off_dpcr', 'away_off_dpcr', 'home_def_dpcr', 'away_def_dpcr', 'home_off_tdcr', 'away_off_tdcr',
          'home_def_tdcr', 'away_def_tdcr', 'home_off_fdcr', 'away_off_fdcr', 'home_def_fdcr', 'away_def_fdcr', 'home_off_rze', 
          'away_off_rze', 'home_def_rze', 'away_def_rze', 'home_def_pts_100', 'away_def_pts_100', 'home_lw_tom', 'away_lw_tom', 
          'home_margin_o_u', 'away_margin_o_u', 'home_margin_ats', 'away_margin_ats', 'point_total', 'vegas_over_under', 
          'hit_over', 'home_min_away_spread', 'vegas_home_min_away_spread', 'home_covered_spread']

col_names.extend(list_1)
col_names.extend(stadium_col_names[0].tolist())
col_names.extend(list_2)
col_names.extend(home_coach_col_names[0].tolist())
col_names.extend(away_coach_col_names[0].tolist())
col_names.extend(list_3)

In [24]:
# create final game data DataFrame
game_data_final = pd.DataFrame(X, columns = col_names)
# export as csv file
game_data_final.to_csv(r'C:\Users\Warren\OneDrive\Documents\NFL Bets\Model\Preprocessed Game Data.csv', 
                       index=False, header=True)

In [40]:
i = 1687
print('Home Team: ' + str(avail_game_data['Home Team'].to_numpy()[i]))
print('Home Pts: ' + str(avail_game_data['Home Pts'].to_numpy()[i]))
print('Away Team: ' + str(avail_game_data['Away Team'].to_numpy()[i]))
print('Away Pts: ' + str(avail_game_data['Away Pts'].to_numpy()[i]))
print('Point Total: ' + str(game_data_final['point_total'].to_numpy()[i]))
print('Vegas OU: ' + str(game_data_final['vegas_over_under'].to_numpy()[i]))
print('Hit Over: ' + str(game_data_final['hit_over'].to_numpy()[i]))
print('Home-Away Spread: ' + str(game_data_final['home_min_away_spread'].to_numpy()[i]))
print('Vegas Home-Away Spread: ' + str(game_data_final['vegas_home_min_away_spread'].to_numpy()[i]))
print('Home Covered Spread: ' + str(game_data_final['home_covered_spread'].to_numpy()[i]))

Home Team: DEN
Home Pts: 14
Away Team: TEN
Away Pts: 16
Point Total: 30.0
Vegas OU: 41.5
Hit Over: 0.0
Home-Away Spread: -2.0
Vegas Home-Away Spread: -3.5
Home Covered Spread: 1.0
